# TSP / Schedule Adherence / Drive Test

## Setup

In [1]:
import os
import sys
import logging
import warnings


root_path = os.path.abspath(f"{os.path.abspath('')}/../../..")

# Show INFO messages and supress auto-reload import warnings
logging.getLogger().setLevel(logging.INFO)
warnings.filterwarnings("ignore")

In [2]:
# Libraries
sys.path.insert(0, f"{root_path}/TSP/tsp-gtfs-realtime")
sys.path.insert(0, f"{root_path}/CDFAndIoT/Service/AssetLibrary")
sys.path.insert(0, f"{root_path}/CDFAndIoT/API")
sys.path.insert(0, f"{root_path}/TSP/ScheduleAdherence/Lambdas/StaticGtfs")
sys.path.insert(0, f"{root_path}/TSP/ScheduleAdherence/Service")
sys.path.insert(0, f"{root_path}/TSP/ScheduleAdherence/DataModel")


In [3]:
%pip install boto3
%load_ext autoreload
%autoreload 2

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pause &> /dev/null
!(cd ../.. && pip install ./tsp-gtfs-realtime) &> /dev/null

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install redis

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Connect directly to Redis cluster
from redis import Redis

REDIS_URL = "10.0.0.38"
REDIS_PORT = "6379"
REDIS_USERNAME = None
REDIS_PASSWORD = None

redis = Redis(
    host=REDIS_URL,
    port=REDIS_PORT,
    ssl=False,
    db=0,
    decode_responses=True,
)

# Check connection
redis.ping()

True

In [7]:
import boto3

from gtt.service.schedule_adherence import AthenaClient, ScheduleAdherenceService
from gtt.service.schedule_adherence.static_gtfs import StaticGtfsService

# Resources and services
athena = boto3.client("athena")
athena_client = AthenaClient(
    athena, output_location="s3://aws-athena-query-results-083011521439-us-east-1/"
)

static_gtfs_service = StaticGtfsService(
    sql_client=athena_client,
    sql_kwargs={"schema": "gtt_gd_gtfs_dev"},
)
schedule_adherence_service = ScheduleAdherenceService(
    redis,
    static_gtfs_service
)

In [8]:
from gtt.data_model.schedule_adherence import RedisChannel

class VehiclePositionsChannel(RedisChannel):
    channel_format = "{agency_name}:{channel_id}:{vehicle_id}"

    @classmethod
    def cache(cls, **kwargs):
        return cls.channel(channel_kwargs={**kwargs, "channel_id": "vehicle_position"})

    @classmethod
    def new(cls, **kwargs):
        return cls.channel(
            channel_kwargs={**kwargs, "channel_id": "new_vehicle_position"}
        )

In [9]:
source = "util/data/vehicle_positions__whole_day.parquet"
polling_rate = None
playback_speed = 5
end_time = None
repeat = False
use_original_time = False
redis_url = REDIS_URL
redis_port = REDIS_PORT
agency_name = "tsptest"
vehicle_list = None
trip_list = None
get_statistics = False

agency_id = "0537199e-e853-11ec-a8b8-f65b686c7d91"

In [10]:
from gtt.data_model.schedule_adherence.static_gtfs import StaticGtfsChannel

redis.publish(StaticGtfsChannel.invalidate(agency_id=agency_id), "")

1

In [11]:
def handle_new_trip_delay(*_, **__):
    print("New trip delay")

pubsub = redis.pubsub()
pubsub.psubscribe(
    **{
        "b'tsp_in_cloud:new_trip_delay:*": handle_new_trip_delay
    }
)

In [14]:
from datetime import datetime

from gtt.data_model.schedule_adherence import ScheduleAdherenceChannel
from gtt.data_model.schedule_adherence import (
    ScheduleStatus,
    RedisChannel,
    VehiclePosition,
)

from tools.recorded_data_publisher import main

checkpoints = []

def callback():
    schedule_statuses = []    
    for key in redis.scan_iter(
        ScheduleAdherenceChannel.cache(agency_id=agency_id)
    ):
        schedule_statuses.append(
            ScheduleStatus.parse_raw(redis.get(key))
        )

    vehicle_positions = []
    for key in redis.scan_iter(
        VehiclePositionsChannel.cache(agency_id=agency_id)
    ):
        gps_fields = ["latitude", "longitude", "speed", "bearing"]
        fields = ["timestamp", "vehicle_id", "trip_id", *gps_fields]
        
        vp = VehiclePosition(**{
                f: redis.hget(key, f)
                for f in fields
            })
        
        vehicle_positions.append(vp)

        redis.publish(
            VehiclePositionsChannel.new(
                agency_name=agency_name,
                vehicle_id=vp.vehicle_id
            ),
            ""
        )

    checkpoints.append({
        "timestamp": datetime.now(),
        "schedule_statuses": schedule_statuses,
        "vehicle_positions": vehicle_positions
    })

In [22]:
# Run test
checkpoints.clear()
start_time = '2022-06-10 15:18:00'
vehicle_list="5518,4172,4114,4127,4137"
vehicle_id_list = vehicle_list.split(",")
main(
    source=source,
    polling_rate=polling_rate,
    playback_speed=playback_speed,
    start_time=start_time,
    end_time=end_time,
    repeat=repeat,
    use_original_time=use_original_time,
    redis_url=redis_url,
    redis_port=redis_port,
    agency_id=agency_id,
    agency_name=agency_name,
    vehicle_list=vehicle_list,
    trip_list=trip_list,
    
    get_statistics=get_statistics,
    callback=callback,
#     api_url="http://gtfs-realtime-test-server.developgtt.com/gtfs-rt/tsptest/vehiclepositions/entities"
#     timezone="America/New_York",
#     begin_with_current_time=True
)

non-unity playback not yet implemented, setting to 1.0
setting start time to 2022-06-10 15:18:00
setting end time to 2022-06-10 23:42:52
adding 132 days 04:28:38.828642 to make times current
Using service
aws_cfg=Config(local_development=True, redis_url='10.0.0.38', redis_port='6379', use_ssl=False, username=None, password=None)


4127
4172
5518
4114
4137
4114
4137
5518
4172
4127
5518
4114
4127
4137
4172
4127
4172
4137
4114
5518
4114
5518
4127
4172
4137


KeyboardInterrupt


In [29]:
redis.publish(f"tsp_in_cloud:tsp_in_cloud:set_tsp_enabled:{agency_id}:3", '{"enabled":true, "vehicle_id":"2"}')

0

In [23]:
# for key in sorted(redis.scan_iter("tsptest:vehicle_position:*")):
#     try:
#         print(redis.hgetall(key))
#         print()
#     except Exception:
#         pass

for key in sorted(redis.scan_iter("*")):
    print(key)

rt_radio_message:device_fields:tsptest-gtfs-realtime-2
rt_radio_message:device_fields:tsptest-gtfs-realtime-3
rt_radio_message:device_fields:tsptest-gtfs-realtime-4
rt_radio_message:tsptest-gtfs-realtime-1:payload
rt_radio_message:tsptest-gtfs-realtime-1:topic
rt_radio_message:tsptest-gtfs-realtime-2:payload
rt_radio_message:tsptest-gtfs-realtime-2:topic
rt_radio_message:tsptest-gtfs-realtime-3:payload
rt_radio_message:tsptest-gtfs-realtime-3:topic
rt_radio_message:tsptest-gtfs-realtime-4:payload
rt_radio_message:tsptest-gtfs-realtime-4:topic
tsptest:last_updated
tsptest:vehicle_ids
tsptest:vehicle_position:1
tsptest:vehicle_position:1102
tsptest:vehicle_position:1104
tsptest:vehicle_position:1107
tsptest:vehicle_position:128
tsptest:vehicle_position:129
tsptest:vehicle_position:136
tsptest:vehicle_position:2
tsptest:vehicle_position:2201
tsptest:vehicle_position:2202
tsptest:vehicle_position:2203
tsptest:vehicle_position:2204
tsptest:vehicle_position:2207
tsptest:vehicle_position:2208

In [ ]:
from gtt.service.schedule_adherence.static_gtfs.static_gtfs_service import (
    StaticGtfsStopTimesChannel
)
from gtt.data_model.schedule_adherence import Schedule

schedules = []
for key in redis.scan_iter(
    StaticGtfsStopTimesChannel.cache()
):
    schedules.append(Schedule.parse_raw(redis.get(key)))

In [ ]:
import json
from gtt.service.schedule_adherence.schedule_adherence_service import ScheduleAdherenceChannel

# tsptest:vehicle_position
# tsp_in_cloud:trip_delay

enabled_vehicle_ids = redis.smembers(tsp_enabled_key)
print(enabled_vehicle_ids)
print()

for key in sorted(redis.scan_iter("tsp_in_cloud:trip_delay*")):
    vehicle_id = ScheduleAdherenceChannel.parse(key)["vehicle_id"]
    if vehicle_id not in enabled_vehicle_ids or vehicle_id not in vehicle_id_list:
        continue

    trip_delay = redis.get(key)
    if trip_delay is None:
        continue
        
    record = json.loads(trip_delay)

    print(key)
    print(record.get("delay") / 60)
    print()

In [ ]:
# for key in sorted(redis.scan_iter("rt_radio_message:*")):
#     print(key)
#     redis.delete(key)
# print(redis.get("rt_radio_message:tsptest-gtfs-realtime-1:payload"))
# redis.get("rt_radio_message:tsptest-gtfs-realtime-1:topic")
# for key in redis.smembers("tsp:tsptest:running_vehicles"):
#     redis.srem("tsp:tsptest:running_vehicles", key)
    
print(redis.smembers("tsp:tsptest:running_vehicles"))

In [ ]:
print(len(checkpoints))
print(len(schedules))

In [ ]:
%pip install pandas

In [ ]:
import pandas as pd
import json
from gtt.data_model.schedule_adherence.static_gtfs import StaticGtfsStop

def flatten_checkpoints(checkpoints, stops):
    vps = []
    sss = []

    for checkpoint in checkpoints:
        timestamp = checkpoint["timestamp"]
        vehicle_positions = checkpoint["vehicle_positions"]
        schedule_statuses = checkpoint["schedule_statuses"]
        
        for vehicle_position in vehicle_positions:
            vps.append({ **vehicle_position.dict(), "timestamp": timestamp,  })

        for schedule_status in schedule_statuses:
            latest_stop_event = schedule_status.stop_events[-1]
            latest_stop = next(
              filter(lambda x: x["stop_id"] == latest_stop_event.stop_id, stops), None
            )

            sss.append({
                **schedule_status.dict(),
                "latitude": latest_stop["latitude"],
                "longitude": latest_stop["longitude"]
            })

    vp_df = pd.DataFrame(
        vps,
        columns=list(VehiclePosition.__fields__.keys())
    )
    ss_df = pd.DataFrame(
        sss,
        columns=[
            "delay",
            "last_updated",
            "next_update",
            "next_stop",
            "exact",
            "latitude",
            "longitude"
        ],
    )
        
    return (ss_df, vp_df)
        
def schedules_stops(schedules):
    stops = dict()
    
    for schedule in schedules:
        for (_, stop_times) in schedule:
            for stop_time in stop_times:
                if stop_time.stop.stop_id not in stops:
                    stops[stop_time.stop.stop_id]= {
                        "stop_id": stop_time.stop.stop_id,
                        "latitude": float(stop_time.stop.stop_lat),
                        "longitude": float(stop_time.stop.stop_lon)
                    }

    return list(stops.values())

stops = schedules_stops(schedules)
ss_df, vp_df = flatten_checkpoints(checkpoints, stops)
stops_df = pd.DataFrame(
    json.loads(
        StaticGtfsStop.__config__.json_dumps(
            stops, default=StaticGtfsStop.__json_encoder__
        )
    ),
    columns=["stop_id", "latitude", "longitude"],
)

In [ ]:
print(stops_df.head())
print(ss_df.head())
print(vp_df.head())

print(vp_df.latitude.min())

In [ ]:
vp_df.drop(["current_status", "stop_id"], axis=1, inplace=True)

In [ ]:
vp_df = vp_df.dropna()
ss_df = ss_df.dropna()
vp_df = vp_df[vp_df["longitude"] > -75]

In [ ]:
import matplotlib.pyplot as plt

def plot_checkpoints(vp_df, ss_df, stops_df):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    fig.set_dpi(300)

    ax.scatter(x=stops_df["latitude"], y=stops_df["longitude"], c="gray", s=0.01)
    ax.scatter(x=vp_df["latitude"], y=vp_df["longitude"], c=vp_df["timestamp"], s=1, cmap="GnBu")
    ax.scatter(x=ss_df["latitude"], y=ss_df["longitude"], c="red", s=0.5)

    ax.set_aspect("equal", adjustable="box")
    plt.show()


plot_checkpoints(vp_df, ss_df, stops_df)